# Graph Analytics
There are some problems which don't neatly fall into the category of traditional "machine learning" but can still be quite powerful.   The problem we will look at today involves relationships between things, and the tool we will use to investigate it is called **graph analytics** or **network analysis**.

To illustrate this, we will start with a fun example: Marvel Comic Book Characters!
![marvel](The_Marvel_Universe.png)


# Read in the data
Graphs are best illustrated by example, so for our data we will use a file containing a list of comic books and the "heroes" that appear in them.   This dataset is from https://www.kaggle.com/csanhueza/the-marvel-universe-social-network.

The data is lines containing "hero" and "comic book".   

In [ ]:

import pandas as pd
heroes_comics = pd.read_csv('/fs/scratch/PAS1585/graphs/heroes_comics.csv')
print(heroes_comics.head(5))

# Forming Links
To link heroes, we will loop over the above dataframe, and do the following:
* Count how often each hero appears
* Keep track of which heroes appear in each individual comic book

Once we have the data for each comic book, we can loop over the comic books and link heroes by the common comic books they appear in.

In [ ]:
from collections import defaultdict
from functools import partial
from itertools import repeat
def nested_defaultdict(default_factory, depth=1):
    result = partial(defaultdict, default_factory)
    for _ in repeat(None, depth - 1):
        result = partial(defaultdict, result)
    return result()

from itertools import combinations 
  
#
# Loop over dataframe, and for each comic, store all of the heroes that appear in that comic
comicHeroList = defaultdict(list)
heroCount = defaultdict(int)
for index, row in heroes_comics.iterrows():
    hero = row['hero']
    comic = row['comic']
    heroCount[hero] += 1
    comicHeroList[comic].append(hero)
#
# Now loop over comics, and count how often heroes show up together
heroPairCount = nested_defaultdict(int,2)
for comic in comicHeroList:
    combos = combinations(comicHeroList[comic],2)
    for (h1,h2) in combos:
        heroPairCount[h1][h2] += 1
        heroPairCount[h2][h1] += 1


# Some printout
Now lets print out the most common hero by count, and then for each of these, print out who they appear together with the most often.

In [ ]:
for hero in sorted(heroCount, key=heroCount.get, reverse=True)[:10]:
    print("Hero: ",hero,"; comic count ",heroCount[hero])
    for hero2 in sorted(heroPairCount[hero], key=heroPairCount[hero].get, reverse=True)[:10]:
        print("   appears with ",hero2,"; number comics ",heroPairCount[hero][hero2])


# Another way to view the data
The tables above are interesting, but it is a little difficult to tell if all of the heroes are just randomnly connected, or if there is some structure to these connections.

There is a way to reveal such structure if it exists.  The idea is to form a network.   A network has (at least) two primary features:
* Nodes:  These are the primary "objects" in our network, and they are somewhat dependent on the problem you are dealing with.   In this case, each "hero" will be a node.  The size or weight of the node will be its total count.
* Edges:  These are the connections between the nodes.   In our case, the edges are the defined by common comic books the heroes appear in.   If two heros appear together in at least one commic book, there will be a single edge between them.  The weight of that edge will be the count of these common appearances.

The python package we will use is **networkx**.  In additon, we will use a community detection algorithm built for networkx called **louvain community detection**.   Before doing anything next, make sure you do the following:
* pip install --user networkx
* pip install --user python-louvain

Then resstart the kernel and rerun the above code blocks.

In [ ]:

import networkx as nx
import community

# Now look for communities
G = nx.Graph()

nodeCountCut = 50.0
edgeCut = 25.0

#
# Now find edges that connect good nodes
numEdges = 0
numGoodEdges = 0
goodEdgeNodes = set()
nodeEdgeCount = defaultdict(int)
for index1 in heroPairCount:
    for index2 in heroPairCount[index1]:
        if index1 != index2:
            numEdges += 1
            if heroPairCount[index1][index2] > edgeCut:
                numGoodEdges += 1
                G.add_edge(index1, index2, weight=heroPairCount[index1][index2])
                goodEdgeNodes.add(index1)
                goodEdgeNodes.add(index2)
                nodeEdgeCount[index1] += 1
                nodeEdgeCount[index2] += 1

#
# Next add to graph only those nodes that actually have at least one connection!
numNodes = 0
numGoodNodes = 0
for hero in heroCount:
    if hero in goodEdgeNodes:
        G.add_node(hero,weight=heroCount[hero],classname=hero)
        numNodes += 1
        if nodeEdgeCount[hero]>0:
            numGoodNodes += 1

print("Total number all nodes  ",numNodes)
print("Total number passing cuts nodes ",numGoodNodes)
print("Total number all edges ",numEdges)
print("Total number good edges ",numGoodEdges)




# Community Detection
Once a network is made, a simple way to look for structure is to see if the nodes cluster.   One of the best tools for this is "louvain community detection" (https://perso.uclouvain.be/vincent.blondel/research/louvain.html).

In [ ]:
#first compute the best partition
# The smaller "resolution" is the more communities you get
resolution = 1.5
partition = community.best_partition(G,weight='weight', resolution=resolution)
print("Number of found communities",len(set(partition.values())))

# Form Lookup Tables
The following shows how to connect the found communities to the original list of heroes.

The resulting tables of the largest hero communities and their members are printed, and the results look really sensible!!

In [ ]:
communityList = defaultdict(list)
communityCount = defaultdict(int)
communityHeroCount = nested_defaultdict(int,2)
for communityID,hero in zip(partition.values(),partition.keys()):
#    print("communityID ",communityID,"; communityIndex ",hero)
    communityList[communityID].append(hero)
    communityCount[communityID] += 1
    communityHeroCount[communityID][hero] = heroCount[hero]

for communityID in sorted(communityCount, key=communityCount.get, reverse=True)[:10]:
    print("Community ID ",communityID,"; number of members ",communityCount[communityID])
    for hero in sorted(communityHeroCount[communityID], key=communityHeroCount[communityID].get, reverse=True)[:10]:
        print("   hero ",hero,"count ",communityHeroCount[communityID][hero])



# Visualizing the Network
The structure revealed above is probably the simplest (and also probably the most valuable) way to reveal information about a network.   One thing that is nice to explore is a visualization of our network, and of the communities that we have found.   Unfortunately, the only thing we have time to explore are tools involving matplotlib, which are fairly limited.  

First some helper functions:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

def community_layout(g, partition):
    """
    Compute the layout for a modular graph.


    Arguments:
    ----------
    g -- networkx.Graph or networkx.DiGraph instance
        graph to plot

    partition -- dict mapping int node -> int community
        graph partitions


    Returns:
    --------
    pos -- dict mapping int node -> (float x, float y)
        node positions

    """

    pos_communities = _position_communities(g, partition, scale=3.)

    pos_nodes = _position_nodes(g, partition, scale=0.75)

    # combine positions
    pos = dict()
    for node in g.nodes():
        pos[node] = pos_communities[node] + pos_nodes[node]

    return pos

def _position_communities(g, partition, **kwargs):

    # create a weighted graph, in which each node corresponds to a community,
    # and each edge weight to the number of edges between communities
    between_community_edges = _find_between_community_edges(g, partition)

    communities = set(partition.values())
    hypergraph = nx.DiGraph()
    hypergraph.add_nodes_from(communities)
    for (ci, cj), edges in between_community_edges.items():
        hypergraph.add_edge(ci, cj, weight=len(edges))

    # find layout for communities
    pos_communities = nx.spring_layout(hypergraph, **kwargs)

    # set node positions to position of community
    pos = dict()
    for node, community in partition.items():
        pos[node] = pos_communities[community]

    return pos

def _find_between_community_edges(g, partition):

    edges = dict()

    for (ni, nj) in g.edges():
        ci = partition[ni]
        cj = partition[nj]

        if ci != cj:
            try:
                edges[(ci, cj)] += [(ni, nj)]
            except KeyError:
                edges[(ci, cj)] = [(ni, nj)]

    return edges

def _position_nodes(g, partition, **kwargs):
    """
    Positions nodes within communities.
    """

    communities = dict()
    for node, community in partition.items():
        try:
            communities[community] += [node]
        except KeyError:
            communities[community] = [node]

    pos = dict()
    for ci, nodes in communities.items():
        subgraph = g.subgraph(nodes)
        pos_subgraph = nx.spring_layout(subgraph, **kwargs)
        pos.update(pos_subgraph)

    return pos

def test():
    # to install networkx 2.0 compatible version of python-louvain use:
    # pip install -U git+https://github.com/taynaud/python-louvain.git@networkx2
    from community import community_louvain

    g = nx.karate_club_graph()
    partition = community_louvain.best_partition(g, resolution=1.5)
    print("Number of found communities",len(set(partition.values())))
    pos = community_layout(g, partition)
    values = [partition.get(node) for node in g.nodes()]
    nx.draw(g, pos, node_color=values)
    plt.show()
    return

# The "Karate Club" example.  
Zachary's Karate Club is a small example network included with networkx which can be used to test a number of features of graph analytics, including community detection.   See this (https://en.wikipedia.org/wiki/Zachary%27s_karate_club) for more details.

Running the "test()" method of the above code, shows this simple graph after community detection.

In [ ]:
test()

# Drawing the Marvel Network
Now lets use the same tool to draw our network.

In [ ]:
from matplotlib import pyplot as plt

#nx.draw_spring(G, cmap = plt.get_cmap('jet'), node_color = values, node_size=30, with_labels=False)
pos = community_layout(G, partition)
values = [partition.get(node) for node in G.nodes()]
nx.draw(G, pos, cmap = plt.get_cmap('jet'), node_color = values, node_size=30, with_labels=False, font_size=6)
plt.show()

## Application of graph analytics to real-world numerical datasets
The above analysis is interesting, but there are not many problems which we will encounter in a typical analysis environment which are like the above Marvel Universe.   It is reasonable to ask if these tools might be applicable in other circumstances.

The idea of community detection seems like one that might have such broader application.   Community detction is just a form of clustering, which itself is an example of "unsupervised" learning.   Can we use community detection in a real work example?  The answer is yes.

To do this we will use "fake" datasets (generated using sklearn tools), as well as some helper functions in the code block below.  The helper fucntions will allow us to visualize the generated datasets.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import pandas as pd
from pandas import DataFrame
import more_itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.impute import SimpleImputer

pd.options.display.max_rows=900
pd.options.display.max_columns=900

import seaborn as sns
from IPython.display import display

import sys, os
import random
import gc

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

def visualize_3d(X,y,labels,algorithm="tsne",title="Data in 3D"):
    from sklearn.manifold import TSNE
    from sklearn.decomposition import PCA
    
    if algorithm=="tsne":
        reducer = TSNE(n_components=3,random_state=47,n_iter=400,angle=0.6)
    elif algorithm=="pca":
        reducer = PCA(n_components=3,random_state=47)
    else:
        raise ValueError("Unsupported dimensionality reduction algorithm given.")
    
    if X.shape[1]>3:
        X = reducer.fit_transform(X)
    else:
        if type(X)==pd.DataFrame:
        	X=X.values
    
    marker_shapes = ["circle","diamond", "circle-open", "square",  "diamond-open", "cross","square-open",]
    traces = []
    for hue in np.unique(y):
        X1 = X[y==hue]

        trace = go.Scatter3d(
            x=X1[:,0],
            y=X1[:,1],
            z=X1[:,2],
            mode='markers',
            name = str(hue),
            marker=dict(
                size=12,
                symbol=marker_shapes.pop(),
                line=dict(
                    width=int(np.random.randint(3,10)/10)
                ),
                opacity=int(np.random.randint(6,10)/10)
            )
        )
        traces.append(trace)


    layout = go.Layout(
        title=title,
        scene=dict(
            xaxis=dict(
                title='Dim 1'),
            yaxis=dict(
                title='Dim 2'),
            zaxis=dict(
                title='Dim 3'), ),
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )
    fig = go.Figure(data=traces, layout=layout)
    iplot(fig)

    
def visualize_3dnew(X,y,labels,algorithm="tsne",title="Data in 3D"):
    from sklearn.manifold import TSNE
    from sklearn.decomposition import PCA
    
    if algorithm=="tsne":
        reducer = TSNE(n_components=3,random_state=47,n_iter=400,angle=0.6)
    elif algorithm=="pca":
        reducer = PCA(n_components=3,random_state=47)
    else:
        raise ValueError("Unsupported dimensionality reduction algorithm given.")
    
    if X.shape[1]>3:
        X = reducer.fit_transform(X)
    else:
        if type(X)==pd.DataFrame:
        	X=X.values
    
    marker_shapes = ["circle","diamond", "circle-open", "square",  "diamond-open", "cross","square-open",]
    DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']
    traces = []
    for hue in np.unique(y):
        X1 = X[y==hue]
        thisText = y[y==hue]
        theseColors = []
        for lbl in labels[y==hue]:
            if lbl>len(DEFAULT_PLOTLY_COLORS):
                color = DEFAULT_PLOTLY_COLORS[0]
            else:
                color = DEFAULT_PLOTLY_COLORS[lbl]
            theseColors.append(color)
            #theseColors.append('rgb(188, 189, 34)')
        trace = go.Scatter3d(
            x=X1[:,0],
            y=X1[:,1],
            z=X1[:,2],
            mode='markers+text',
            text=thisText,
            name = str(hue),
            marker=dict(
                size=12,
                symbol=marker_shapes.pop(),
                line=dict(
                    width=int(np.random.randint(3,10)/10)
                ),
                color=theseColors,
                opacity=int(np.random.randint(6,10)/10)
            )
        )
        traces.append(trace)


    layout = go.Layout(
        title=title,
        scene=dict(
            xaxis=dict(
                title='Dim 1'),
            yaxis=dict(
                title='Dim 2'),
            zaxis=dict(
                title='Dim 3'), ),
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )
    fig = go.Figure(data=traces, layout=layout)
    iplot(fig)

    
def visualize_2d(X,y,algorithm="tsne",title="Data in 2D",figsize=(8,8)):
    from sklearn.manifold import TSNE
    from sklearn.decomposition import PCA
    if algorithm=="tsne":
        reducer = TSNE(n_components=2,random_state=47,n_iter=400,angle=0.6)
    elif algorithm=="pca":
        reducer = PCA(n_components=2,random_state=47)
    else:
        raise ValueError("Unsupported dimensionality reduction algorithm given.")
    if X.shape[1]>2:
        X = reducer.fit_transform(X)
    else:
        if type(X)==pd.DataFrame:
        	X=X.values
    f, (ax1) = plt.subplots(nrows=1, ncols=1,figsize=figsize)
    sns.scatterplot(X[:,0],X[:,1],hue=y,ax=ax1);
    ax1.set_title(title);
    plt.show();


## Generating Fake Data
sklearn comes with a powerful tool for generating datasets call "make_classification".   The resulting generated datasets can then be used to test other algorithms such as classification and regression.   We will use this to test the clustering ability of graph analytics algorithms.

The important parameters for "make_classification" are the following:
* n_samples: this is the total amount of data you want generated
* n_classes: the total number of different classes you want your data to have
* n_features: the total number of features that describe each data point in your sample
* class_sep: the larger this value, the more your classes are separated (and presumably the easier they are to classify or cluster)

In [ ]:
from sklearn.datasets import make_classification
import pandas as pd
X,Y = make_classification(n_samples=1000, n_features=10,
                    n_redundant=1, n_repeated=0, n_classes=4, n_clusters_per_class=1,
                    class_sep=1.2,
                   flip_y=0)
Xd = pd.DataFrame(X)
Yd = pd.Series(Y)
visualize_3dnew(Xd,Yd,Yd)

# Number of data points per class

In [ ]:

from collections import defaultdict
from functools import partial
from itertools import repeat
def nested_defaultdict(default_factory, depth=1):
    result = partial(defaultdict, default_factory)
    for _ in repeat(None, depth - 1):
        result = partial(defaultdict, result)
    return result()


classNums = defaultdict(int)
for y in Y:
    classNums[y] += 1
print("Class numbers ",classNums)


# Edges
We need some way to define edges, or connect, our datapoints.   To do this, we will use a concept we introduced previously: **cosine similarity**.   Our points each lie in an n-dimensional space (defined by the features).   By calculating the cosine between each point, we can define a strength of the connection:
* if the cosine>threshold then the points are connected
* if the cosing<threshold, then the points are not connected

We are assuming that since the data is generated to come from different classes, that the cosine of points ffrom the same classes should be closer than points from different classes.

In [ ]:

from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

simSame = []
simDifferent = []


labels = []
index = 0
for y in Y:
    labels.append(index)
    index += 1

edges = nested_defaultdict(float,2)
for x1,y1,index1 in zip(X,Y,labels):
    for x2,y2,index2 in zip(X,Y,labels):
        if index2>index1:
            sim = cosine_similarity(x1,x2)
            edges[index1][index2] = sim
            edges[index2][index1] = sim
            if y1==y2:
                simSame.append(sim)
            else:
                simDifferent.append(sim)



# Compare cosing similarity for points
Look at points of the same class, versus points of different classes.

In [ ]:
from matplotlib import pyplot
import numpy as np

bins = np.linspace(-1, 1, 100)

pyplot.hist(simSame, bins, alpha=0.5, label='same')
pyplot.hist(simDifferent, bins, alpha=0.5, label='different')
pyplot.legend(loc='upper right')
pyplot.show()

# Define the connections
From the curve above, it looks like we can call connected points, those that have cosine>0.5.

With this definition, we can form a graph.
* Edges are nodes with have cosine>threshold (which we set to be 0.5)
* Nodes are those which have at least some threshold of connections

In [ ]:

import networkx as nx
import community

# Now look for communities
G = nx.Graph()

nodeCountCut = 5.0
edgeCut = 0.5

#
# Now find edges that connect good nodes
numEdges = 0
numGoodEdges = 0
goodEdgeNodes = set()
nodeEdgeCount = defaultdict(int)
for index1 in edges:
    for index2 in edges[index1]:
        if index1 != index2:
            numEdges += 1
            if edges[index1][index2] > edgeCut:
                numGoodEdges += 1
                G.add_edge(index1, index2, weight=edges[index1][index2])
                goodEdgeNodes.add(index1)
                goodEdgeNodes.add(index2)
                nodeEdgeCount[index1] += 1
                nodeEdgeCount[index2] += 1

#
# Next add to graph only those nodes that actually have at least one connection!
numNodes = 0
numGoodNodes = 0
for x,y,index in zip(X,Y,labels):
    G.add_node(index,weight=nodeEdgeCount[index],classname=y)
    numNodes += 1
    if nodeEdgeCount[index]>0:
        numGoodNodes += 1

print("Total number all nodes  ",numNodes)
print("Total number passing cuts nodes ",numGoodNodes)
print("Total number all edges ",numEdges)
print("Total number good edges ",numGoodEdges)




# Run Community Detection

In [ ]:
#first compute the best partition
# The smaller "resolution" is the more communities you get
resolution = 1.0
partition = community.best_partition(G,weight='weight', resolution=resolution)
print("Number of found communities",len(set(partition.values())))


# Connect Communities to Classes
The found communities are the same as the number of classes.   Remember: this was totally unsupervised!   Now we need to see if the communities actually correspond to the true classes.

This is a little more complicated than the marvel universe.  In our case, each point already belowngs to a class, and we want to know how often our classes are connected to the same community.

In [ ]:
indexToCommunity = {}
for communityID,communityIndex in zip(partition.values(),partition.keys()):
    indexToCommunity[communityIndex] = communityID
    
communityClassCount = nested_defaultdict(int,2)
classCommunityCount = nested_defaultdict(int,2)
communityAssignment = []
for x,y,index in zip(X,Y,labels):
    if index in indexToCommunity:
#        print("Data label ",index,"; class ",y)
        community = indexToCommunity[index]
#        print("Data label ",index,"; class ",y,"; assigned community ",community)
        communityClassCount[community][y] += 1
        classCommunityCount[y][community] += 1
        communityAssignment.append(community)
communityAssignment = np.asarray(communityAssignment)
print()
print("Top classes for each community")
for community in communityClassCount:
    print("community",community)
    for y in communityClassCount[community]:
        print("   class ",y," count ",communityClassCount[community][y])
        
print()
print("Top communities for each class")
for y in classCommunityCount:
    print("class",y)
    for community in classCommunityCount[y]:
        print("   community ",community," count ",classCommunityCount[y][community])
        


# Now visualize

In [ ]:
visualize_3dnew(Xd,Yd,communityAssignment)

In [ ]:
from matplotlib import pyplot as plt

#nx.draw_spring(G, cmap = plt.get_cmap('jet'), node_color = values, node_size=30, with_labels=False)
pos = community_layout(G, partition)
values = [partition.get(node) for node in G.nodes()]
nx.draw(G, pos, cmap = plt.get_cmap('jet'), node_color = values, node_size=30, with_labels=False, font_size=6)
plt.show()

In [ ]:
values = [partition.get(node) for node in G.nodes()]

nx.draw_spring(G, cmap = plt.get_cmap('jet'), node_color = values, node_size=30, with_labels=False)

# Extra Credit (4 pts total)
Now let's apply this to a data sample we used previously: pulsars.   We I want you to do is to 
* calculate the cosine similarity among all points in the pulsar dataset
* plot the cosine similary for true pulsars (one class) vs non-pulsars (the other class).
* Run community detection and see how well the communities line up with the true classes
* Plot the communities using visualize_3dnew

The code below sets things up by reading the data in!

In [ ]:
import pandas as pd

#
# Read in all of the other digits
fname = 'https://raw.githubusercontent.com/big-data-analytics-physics/data/master/HTRU2/HTRU_2a.csv'
dfAll = pd.read_csv(fname)
print(dfAll.head(5))
#
# The data already has a 0/1 class variable that defines signal (1) and background (0)
#
# The data is already combined but it will be usefull to split it so we can look at 
# signal and background separately.
dfA = dfAll[dfAll['class']==1]
dfB = dfAll[dfAll['class']==0]

print("Length of signal sample:     ",len(dfA))
print("Length of background sample: ",len(dfB))

#
# Shuffle the data here
from sklearn.utils import shuffle
dfBShuffle = shuffle(dfB)
#
# Uncomment the next line to limit dfB to be the same length as dfA
#dfB_use = dfBShuffle
dfB_use = dfBShuffle.head(len(dfA))


dfCombined = dfB_use
dfCombined = pd.concat([dfCombined, dfA])
dfCombined = shuffle(dfCombined)

print("Size of signal sample ",len(dfA))
print("Size of background sample ",len(dfB_use))
print("Size of combined sample ",len(dfCombined))

from sklearn.utils import shuffle
dfCombinedShuffle = shuffle(dfCombined,random_state=42)    # by setting the random state we will get reproducible results

X = dfCombinedShuffle.as_matrix(columns=dfCombinedShuffle.columns[:8])
Y = dfCombinedShuffle['class'].values

